In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
import pandas as pd

In [28]:
!pip install git+https://github.com/csebuetnlp/normalizer

  Cloning https://github.com/csebuetnlp/normalizer to /tmp/pip-req-build-5rmnjanf
  Running command git clone --filter=blob:none --quiet https://github.com/csebuetnlp/normalizer /tmp/pip-req-build-5rmnjanf
  Resolved https://github.com/csebuetnlp/normalizer to commit d405944dde5ceeacb7c2fd3245ae2a9dea5f35c9
  Preparing metadata (setup.py) ... done


In [29]:
from transformers import pipeline
from normalizer import normalize
import torch

MODEL_PATH = "/content/drive/MyDrive/banglabert_sentiment"

pipe = pipeline(
    "sentiment-analysis",
    model=MODEL_PATH,
    tokenizer=MODEL_PATH,
    return_all_scores=True,
    device=0
    if torch.cuda.is_available()
    else -1
)

Device set to use cpu
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [30]:
label_map = {
    0: "Neutral",
    1: "Positive",
    2: "Negative"
}

reverse_label_map = {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
}

def predict_with_confidence(text):
    text = normalize(text)
    outputs = pipe(text)[0]
    best = max(outputs, key=lambda x: x['score'])
    pred_label = reverse_label_map[best['label']]
    confidence = best['score']
    return pred_label, confidence

In [31]:
def evaluate_dataset(df, dataset_name="Test Set"):

    results = []
    for i in range(len(df)):
        text = df.loc[i, "text"]
        true_label = df.loc[i, "label"]
        pred_label, confidence = predict_with_confidence(text)

        results.append({
            "text": text,
            "true_label": true_label,
            "pred_label": pred_label,
            "confidence": confidence,
            "correct": pred_label == true_label
        })

    results_df = pd.DataFrame(results)

    # Calculate metrics
    accuracy = results_df["correct"].mean()
    avg_confidence = results_df["confidence"].mean()
    correct_conf = results_df[results_df["correct"] == True]["confidence"].mean()
    incorrect_conf = results_df[results_df["correct"] == False]["confidence"].mean()

    overconfident = results_df[(results_df["correct"] == False) & (results_df["confidence"] >= 0.9)]
    overconfidence_rate = len(overconfident) / len(results_df)

    low_conf_correct = results_df[(results_df["correct"] == True) & (results_df["confidence"] < 0.6)]
    low_confidence_rate = len(low_conf_correct) / len(results_df)

    neutral_shift = results_df[
        ((results_df["true_label"] == 1) | (results_df["true_label"] == 2)) &
        (results_df["pred_label"] == 0)
    ]
    neutral_shift_rate = len(neutral_shift) / len(results_df)


    # Print results
    print(f"{dataset_name.upper()} EVALUATION RESULTS")
    print(f"Total Samples: {len(results_df)}")
    print()
    print(f"Overall Accuracy: {accuracy*100:.2f}%")
    print(f"Average Confidence: {avg_confidence:.4f}")
    print(f"Average Confidence (Correct Predictions): {correct_conf:.4f}")
    print(f"Average Confidence (Incorrect Predictions): {incorrect_conf:.4f}")
    print(f"Overconfidence Rate (Wrong + Confidence >= 0.9): {overconfidence_rate*100:.2f}%")
    print(f"Low Confidence Rate (Correct + Confidence < 0.6): {low_confidence_rate*100:.2f}%")
    print(f"Neutral Shift Rate: {neutral_shift_rate*100:.2f}%")

In [32]:
#A1

In [33]:
adv_df = pd.read_csv("a1_adv_test.csv")
adv_df.rename(columns={"Data": "text", "Label": "label"}, inplace=True)
adv_results = evaluate_dataset(adv_df, dataset_name="Sarcasm Injection Attack")

SARCASM INJECTION ATTACK EVALUATION RESULTS
Total Samples: 636

Overall Accuracy: 52.52%
Average Confidence: 0.7993
Average Confidence (Correct Predictions): 0.8635
Average Confidence (Incorrect Predictions): 0.7284
Overconfidence Rate (Wrong + Confidence >= 0.9): 18.24%
Low Confidence Rate (Correct + Confidence < 0.6): 3.77%
Neutral Shift Rate: 12.42%


In [34]:
#A2

In [35]:
adv_df = pd.read_csv("a2_adv_test.csv")
adv_df.rename(columns={"Data": "text", "Label": "label"}, inplace=True)
adv_results = evaluate_dataset(adv_df, dataset_name="Ancholik Substitution Attack")

ANCHOLIK SUBSTITUTION ATTACK EVALUATION RESULTS
Total Samples: 1585

Overall Accuracy: 63.97%
Average Confidence: 0.8119
Average Confidence (Correct Predictions): 0.8440
Average Confidence (Incorrect Predictions): 0.7548
Overconfidence Rate (Wrong + Confidence >= 0.9): 15.52%
Low Confidence Rate (Correct + Confidence < 0.6): 6.50%
Neutral Shift Rate: 7.70%


In [36]:
#A3

In [37]:
adv_df = pd.read_csv("a3_adv_test.csv")
adv_df.rename(columns={"Data": "text", "Label": "label"}, inplace=True)
adv_results = evaluate_dataset(adv_df, dataset_name="Bangla-English Mixing Attack")

BANGLA-ENGLISH MIXING ATTACK EVALUATION RESULTS
Total Samples: 942

Overall Accuracy: 67.62%
Average Confidence: 0.8060
Average Confidence (Correct Predictions): 0.8448
Average Confidence (Incorrect Predictions): 0.7247
Overconfidence Rate (Wrong + Confidence >= 0.9): 9.77%
Low Confidence Rate (Correct + Confidence < 0.6): 6.37%
Neutral Shift Rate: 13.59%


In [38]:
#A4

In [39]:
adv_df = pd.read_csv("a4_adv_test.csv")
adv_df.rename(columns={"Data": "text", "Label": "label"}, inplace=True)
adv_results = evaluate_dataset(adv_df, dataset_name="Implicit Negation Attack")

IMPLICIT NEGATION ATTACK EVALUATION RESULTS
Total Samples: 998

Overall Accuracy: 69.34%
Average Confidence: 0.8057
Average Confidence (Correct Predictions): 0.8467
Average Confidence (Incorrect Predictions): 0.7130
Overconfidence Rate (Wrong + Confidence >= 0.9): 9.12%
Low Confidence Rate (Correct + Confidence < 0.6): 7.01%
Neutral Shift Rate: 9.92%


In [40]:
#A5

In [41]:
adv_df = pd.read_csv("a5_adv_test.csv")
adv_df.rename(columns={"Data": "text", "Label": "label"}, inplace=True)
adv_results = evaluate_dataset(adv_df, dataset_name="Discourse Marker Dilution Attack")

DISCOURSE MARKER DILUTION ATTACK EVALUATION RESULTS
Total Samples: 1407

Overall Accuracy: 70.65%
Average Confidence: 0.8102
Average Confidence (Correct Predictions): 0.8419
Average Confidence (Incorrect Predictions): 0.7339
Overconfidence Rate (Wrong + Confidence >= 0.9): 9.59%
Low Confidence Rate (Correct + Confidence < 0.6): 7.04%
Neutral Shift Rate: 11.37%


In [42]:
#A6

In [43]:
adv_df = pd.read_csv("a6_adv_test.csv")
adv_df.rename(columns={"Data": "text", "Label": "label"}, inplace=True)
adv_results = evaluate_dataset(adv_df, dataset_name="Emoji Polarity Conflict Attack")

EMOJI POLARITY CONFLICT ATTACK EVALUATION RESULTS
Total Samples: 1586

Overall Accuracy: 69.36%
Average Confidence: 0.8403
Average Confidence (Correct Predictions): 0.8677
Average Confidence (Incorrect Predictions): 0.7783
Overconfidence Rate (Wrong + Confidence >= 0.9): 15.26%
Low Confidence Rate (Correct + Confidence < 0.6): 4.98%
Neutral Shift Rate: 5.86%


In [44]:
#A7

In [45]:
adv_df = pd.read_csv("a7_adv_test.csv")
adv_df.rename(columns={"Data": "text", "Label": "label"}, inplace=True)
adv_results = evaluate_dataset(adv_df, dataset_name="Opinion Holder Shift Attack")

OPINION HOLDER SHIFT ATTACK EVALUATION RESULTS
Total Samples: 1500

Overall Accuracy: 36.33%
Average Confidence: 0.8148
Average Confidence (Correct Predictions): 0.8383
Average Confidence (Incorrect Predictions): 0.8014
Overconfidence Rate (Wrong + Confidence >= 0.9): 35.53%
Low Confidence Rate (Correct + Confidence < 0.6): 4.33%
Neutral Shift Rate: 15.00%


In [46]:
#A8

In [47]:
adv_df = pd.read_csv("a8_adv_test.csv")
adv_df.rename(columns={"Data": "text", "Label": "label"}, inplace=True)
adv_results = evaluate_dataset(adv_df, dataset_name="Metaphor Substitution Attack")

METAPHOR SUBSTITUTION ATTACK EVALUATION RESULTS
Total Samples: 1578

Overall Accuracy: 70.28%
Average Confidence: 0.8188
Average Confidence (Correct Predictions): 0.8539
Average Confidence (Incorrect Predictions): 0.7359
Overconfidence Rate (Wrong + Confidence >= 0.9): 10.90%
Low Confidence Rate (Correct + Confidence < 0.6): 6.84%
Neutral Shift Rate: 6.34%
